In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
train_datagen_nor = ImageDataGenerator(validation_split=0.15,
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen_nor.flow_from_directory(r'/content/drive/MyDrive/n+c+g/train',target_size=(512, 512), subset='training',
                                                       batch_size=50, class_mode='categorical')
valid_data = train_datagen_nor.flow_from_directory(r'/content/drive/MyDrive/n+c+g/train',target_size=(512, 512), 
                                                    subset='validation'   ,batch_size=64, class_mode='categorical')


Found 6604 images belonging to 50 classes.
Found 1150 images belonging to 50 classes.


In [15]:
from keras.regularizers import l2
from keras.layers import BatchNormalization
from keras.layers import GaussianNoise
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 8, kernel_size = (3,3), kernel_regularizer=l2(0.01), padding = 'Same', activation ='relu', input_shape = (512,512,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters = 48, kernel_size = (3,3), kernel_regularizer=l2(0.01), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides = 2))
model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),kernel_regularizer=l2(0.01), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), kernel_regularizer=l2(0.01), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), kernel_regularizer=l2(0.01), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation = "relu", kernel_regularizer=l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(50, activation = "relu", kernel_regularizer=l2(0.01)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(50, activation = "softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 512, 512, 8)       224       
_________________________________________________________________
batch_normalization_3 (Batch (None, 512, 512, 8)       32        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 512, 512, 48)      3504      
_________________________________________________________________
batch_normalization_4 (Batch (None, 512, 512, 48)      192       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 256, 256, 48)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 256, 256, 128)     55424     
_________________________________________________________________
batch_normalization_5 (Batch (None, 256, 256, 128)    

In [4]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

csv = CSVLogger("/content/drive/MyDrive/n+c+g/normal+corrupted+gauss-model_history_log.csv", append=True)
es1= tf.keras.callbacks.EarlyStopping( monitor='val_accuracy',patience=2, restore_best_weights=True)
mc = ModelCheckpoint('/content/drive/MyDrive/n+c+g/normal+corrupted+gauss--cnnmodel.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [16]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, decay = 0.9)
model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [7]:
history = model.fit(train_data,epochs=100,shuffle=True,validation_data=valid_data,callbacks=[csv,es1,mc])

Epoch 1/100
133/133 [==============================] - 101s 756ms/step - loss: 2.0988 - accuracy: 0.7936 - val_loss: 12.0994 - val_accuracy: 0.0200

Epoch 00001: val_accuracy did not improve from 0.03391
Epoch 2/100
133/133 [==============================] - 101s 756ms/step - loss: 1.9805 - accuracy: 0.8422 - val_loss: 11.3954 - val_accuracy: 0.0200

Epoch 00002: val_accuracy did not improve from 0.03391
Epoch 3/100
133/133 [==============================] - 100s 750ms/step - loss: 1.9105 - accuracy: 0.8607 - val_loss: 7.8898 - val_accuracy: 0.0652

Epoch 00003: val_accuracy improved from 0.03391 to 0.06522, saving model to /content/drive/MyDrive/n+c+g/normal+corrupted+gauss--cnnmodel.h5
Epoch 4/100
133/133 [==============================] - 99s 741ms/step - loss: 1.8514 - accuracy: 0.8840 - val_loss: 4.1637 - val_accuracy: 0.0930

Epoch 00004: val_accuracy improved from 0.06522 to 0.09304, saving model to /content/drive/MyDrive/n+c+g/normal+corrupted+gauss--cnnmodel.h5
Epoch 5/100
133

In [10]:
model.save_weights(r'/content/drive/MyDrive/n+c+g/normal+corrupted-cnn-model-weights.h5')

In [17]:
model.load_weights('normal+corrupted-cnn-model-weights1.h5')

In [8]:
train_acc=model.evaluate(train_data)
val_acc = model.evaluate(valid_data)
print("Train_Accuracy: %.2f%%" % (train_acc[1]*100))
print("valid_Accuracy: %.2f%%" % (val_acc[1]*100))

18/18 [==============================] - 14s 787ms/step - loss: 1.7287 - accuracy: 0.9774
Train_Accuracy: 98.41%
valid_Accuracy: 97.74%


In [6]:

test_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
test_genrator_nor = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-30',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 278s 15s/step - loss: 9.3827 - accuracy: 0.8877
Test_Accuracy: 88.77%


In [19]:
test_genrator_nor10 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-10',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor10)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 272s 15s/step - loss: 8.9635 - accuracy: 0.9838
Test_Accuracy: 98.38%


In [21]:
test_genrator_nor20 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-20',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor20)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 273s 15s/step - loss: 9.0785 - accuracy: 0.9762
Test_Accuracy: 97.62%


In [22]:

test_genrator_nor25 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-25\test',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor25)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1050 images belonging to 50 classes.
14/14 [==============================] - 219s 16s/step - loss: 9.0020 - accuracy: 0.9771
Test_Accuracy: 97.71%


In [24]:
test_genrator_nor40 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-40',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor40)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 280s 16s/step - loss: 10.0224 - accuracy: 0.6608
Test_Accuracy: 66.08%


In [28]:
test_genrator_nor40 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-40',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor40)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 280s 16s/step - loss: 10.0224 - accuracy: 0.6208
Test_Accuracy: 62.08%


In [25]:
test_genrator_nor50 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-50\test',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor50)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 282s 16s/step - loss: 10.5427 - accuracy: 0.2954
Test_Accuracy: 29.54%


In [26]:

test_genrator_normal = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\dataset-50\test',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_normal)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 232s 13s/step - loss: 8.6591 - accuracy: 0.9846
Test_Accuracy: 98.46%


In [29]:
test_genrator_gau = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\gaussian-50\test',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_gau)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 232s 13s/step - loss: 8.6591 - accuracy: 0.5105
Test_Accuracy: 51.05%


In [30]:
test_genrator_nor60 = test_datagen.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-60',
        target_size=(512, 512),
        batch_size=75,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor60)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
18/18 [==============================] - 232s 13s/step - loss: 28.6591 - accuracy: 0.0165
Test_Accuracy: 1.65%
